In [1]:
import os

current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")
os.chdir("/workspace")
print(f"Working directory changed to: {os.getcwd()}")

Current working directory: /workspace/notebooks
Working directory changed to: /workspace


In [2]:
from datasets import load_dataset, load_from_disk
import verifiers as vf
from verifiers.envs.bfcl_inthinking_env import BfclITEnv
import json
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

[2025-04-19 17:50:49,050] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


# Loading Env

In [3]:
env = BfclITEnv(
    dataset="bfcl",
    tools=[],
    max_num_turns=1,
    max_steps_per_turn=10,
    curriculum_learning=True,
    use_latest_trl=False,
)

2025-04-19 17:50:51.186 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:186 - Initializing MultiStepEnv
2025-04-19 17:50:51.187 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:228 - Initializing Scoring Rubric
2025-04-19 17:50:51.187 | INFO     | verifiers.envs.bfcl_inthinking_env:__init__:230 - Initializing LLM + Env Parsers


In [4]:
ds_train = env.get_dataset(max_num_turns=1)
ds_eval = env.get_eval_dataset(max_num_turns=1, max_turn_only=True)
ds_train

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/379 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/745 [00:00<?, ? examples/s]

Filter:   0%|          | 0/366 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'initial_config', 'path', 'involved_classes', 'num_turns', 'answer', 'prompt', 'user_question_bank', 'ground_truth_bank', '__index_level_0__'],
    num_rows: 100
})

In [5]:
ds_train[0]

{'id': 'multi_turn_base_51',
 'question': '[[{"role": "user", "content": "I\'ve noticed that some of my car doors are slightly ajar while others seem to be securely locked. Would you be able to verify and make sure all doors are properly locked for safety?"}]]',
 'initial_config': '{"VehicleControlAPI": {"fuelLevel": 15.5, "batteryVoltage": 12.8, "engineState": "stopped", "doorStatus": {"driver": "unlocked", "passenger": "unlocked", "rear_left": "unlocked", "rear_right": "unlocked"}, "acTemperature": 22.0, "fanSpeed": 60, "acMode": "auto", "humidityLevel": 45.0, "headLightStatus": "off", "parkingBrakeStatus": "released", "parkingBrakeForce": 0.0, "slopeAngle": 0.0, "distanceToNextVehicle": 100.0, "cruiseStatus": "inactive", "destination": "None", "frontLeftTirePressure": 35.0, "frontRightTirePressure": 35.0, "rearLeftTirePressure": 33.0, "rearRightTirePressure": 33.0}}',
 'path': ['VehicleControlAPI.displayCarStatus',
  'VehicleControlAPI.lockDoors',
  'VehicleControlAPI.startEngine',


In [6]:
rubric = env.get_rubric()
training_args = vf.get_default_grpo_config(run_name="debug", num_gpus=2)
training_args.num_train_epochs = 1
training_args.num_generations = 2
training_args.max_completion_length = 2048
training_args.report_to = "none"
training_args.per_device_train_batch_size = 2
training_args.gradient_accumulation_steps = 4
training_args.num_iterations = 2
training_args.beta = 0.001
training_args.max_grad_norm = 0.2
training_args.eval_strategy = "steps"
training_args.eval_on_start = False
training_args.eval_steps = 50
training_args.save_strategy = "steps"
training_args.save_steps = 100
training_args.per_device_eval_batch_size = 2
training_args.eval_accumulation_steps = 1
training_args.data_seed = 42
training_args.sync_ref_model = True
training_args.ref_model_mixup_alpha = 1.0
training_args.ref_model_sync_steps = 100

In [7]:
#model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model_name="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model, tokenizer = vf.get_model_and_tokenizer(model_name)

trainer = vf.GRPOEnvTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=rubric,
    env=env,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    debug_generate=True,
    debug_rewards=True,
    model_name=model_name,
    run_name="debug_baseline",
    use_dr_grpo=False,
    test_hypothesis_clip_advantage=False,
    apply_overlong_filtering=True,
    print_sample_completions=True,
)

Using Liger kernel


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

Applied Liger kernels to Qwen2


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


...

INFO 04-19 17:52:52 weight_utils.py:254] Using model weights format ['*.safetensors']
INFO 04-19 17:52:53 weight_utils.py:304] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-19 17:52:53 model_runner.py:1115] Loading model weights took 3.3414 GB
INFO 04-19 17:52:57 worker.py:267] Memory profiling takes 3.29 seconds
INFO 04-19 17:52:57 worker.py:267] the current vLLM instance can use total_gpu_memory (23.55GiB) x gpu_memory_utilization (0.70) = 16.48GiB
INFO 04-19 17:52:57 worker.py:267] model weights take 3.34GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 8.07GiB; the rest of the memory reserved for KV Cache is 4.99GiB.
INFO 04-19 17:52:57 executor_base.py:111] # cuda blocks: 11682, # CPU blocks: 9362
INFO 04-19 17:52:57 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 1.43x
INFO 04-19 17:53:01 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_ut

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:15<00:00,  2.21it/s]

INFO 04-19 17:53:17 model_runner.py:1562] Graph capturing finished in 16 secs, took 0.22 GiB
INFO 04-19 17:53:17 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 23.07 seconds


In [ ]:
#trainer.train()

# Evaluating Outputs

In [9]:
eval_ds = load_from_disk(
    "notebooks/tmp_multiturn_example_rl/eval_result_Qwen2.5_1.5B_Instruct_step_402.hf"
)
eval_ds

Dataset({
    features: ['step', 'prompt', 'completion', 'reward', 'correctness', 'contains_gibberish', 'dataset_rows', 'train'],
    num_rows: 200
})

In [10]:
example = eval_ds[0]
example

{'step': 402,
 'prompt': 'First User Request: "I\'m planning a business class trip from JFK in New York to LAX in Los Angeles on December 15, 2024. Alex Johnson will be my traveling companion. I intend to use my credit card with label \'id_1234\' to cover the $4500 trip cost. I\'ve got my access token here: ABCD1234. Once booked, I\'ll need to cancel the trip immediately due to unexpected changes in my schedule."\n\nInvolved Classes: [\'TicketAPI\', \'TravelAPI\']\n\nFunction Documentation:\n\nTools for TicketAPI: [\'close_ticket\', \'create_ticket\', \'edit_ticket\', \'get_ticket\', \'get_user_tickets\', \'logout\', \'resolve_ticket\', \'ticket_get_login_status\', \'ticket_login\']\n\nTools for TravelAPI: [\'authenticate_travel\', \'book_flight\', \'cancel_booking\', \'compute_exchange_rate\', \'contact_customer_support\', \'get_all_credit_cards\', \'get_budget_fiscal_year\', \'get_credit_card_balance\', \'get_flight_cost\', \'get_nearest_airport_by_city\', \'list_all_airports\', \'pu

In [19]:
trainer.sampling_params

SamplingParams(n=2, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.9, top_p=1.0, top_k=50, min_p=0.0, seed=None, stop=[], stop_token_ids=[], bad_words=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=2048, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None, guided_decoding=None)

In [16]:
output = trainer.env.generate(
    prompts=[example["dataset_rows"]["prompt"]],
    llm=trainer.llm,
    sampling_params=trainer.sampling_params,
    dataset_rows=[example["dataset_rows"]],
    debug=True,  # True for detailed output
)

Initial User Prompt: You are an expert in composing functions. You are given a question from a user and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to complete the task.
You have access to the following tools to help solve the task:

{"name": "close_ticket", "description": "Close a ticket.", "parameters": {"type": "dict", "properties": {"ticket_id": {"type": "integer", "description": "ID of the ticket to be closed. "}}, "required": ["ticket_id"]}, "response": {"type": "dict", "properties": {"status": {"type": "string", "description": "Status of the close operation."}}}}
{"name": "create_ticket", "description": "Create a ticket in the system and queue it.", "parameters": {"type": "dict", "properties": {"title": {"type": "string", "description": "Title of the ticket."}, "description": {"type": "string", "description": "Description of the ticket. Defaults to an empty string.", "default": ""}, "priority": {"type": "integer", "de

In [36]:
output = trainer.env.generate(
    prompts=[example["dataset_rows"]["prompt"]],
    llm=trainer.llm,
    sampling_params=trainer.sampling_params,
    dataset_rows=[example["dataset_rows"]],
    debug=False,  # True for detailed output
)

In [12]:
output["messages"]

[[{'role': 'assistant',
   'content': 'Alright, let\'s tackle this step by step. The user wants to book a business class trip from JFK to LAX on December 15, 2024. They\'re using a credit card labeled \'id_1234\' with an access token ABCD1234. Once booked, they need to cancel immediately.\n\nFirst, I\'ll need to check if the user has access to the travel information. They mentioned having an access token, so I should verify if the authentication endpoint (\'authenticate\') exists and has the necessary parameters. The parameters required here are client_id, client_secret, refresh_token, grant_type, user_first_name, and user_last_name. Looking back at the tools, I see that \'get_user_tickets\' might not be relevant here since it\'s for ticket reviews. However, \'get_current_user\' is part of the authentication tools and might handle this. So, the first step is to call \'get_current_user\' using the access token. The response will give the user\'s details and whether they are authenticate

In [15]:
print(trainer.tokenizer.decode(output["ids"][0]))

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


json
{
  "step-by-step思考过程": [
    {
      "reasoning": {
        "required": ["access_token", "card_id", "travel_date", "travel_from", "travel_to", "travel_class"],
        "response": {"status": "success"}
      }
    },
    {
      "tool": {
        "name": "get_user_tickets",
        "args": { "required": ["user_first_name", "user_last_name"] },
        "response": {
          "status": "success",
          "properties": {
            "id": {"type": "string", "description": "The ID of the booking"}
          }
        }
      }
    },
    {
      "tool": {
        "name": "create_ticket",
        "args": { "ticket_id": {"type": "string", "description": "The ID of the booking" }, "updates": {"type": "dict", "description": "Dictionary containing the fields to be updated." }}
      },
      "output": {
        "status": "success",
        "properties": {
          "id": {"type": "string", "description": "The ID of the booking"}
        }
      }
    },
    {
      "tool": "cancel_book

In [14]:
len(output["ids"][0])

537

In [49]:
len(output["mask"][0])

1722

In [51]:
masked = []
for x, y in zip(output["ids"][0], output["mask"][0]):
    if y == 1:
        masked.append(x)
    else:
        masked.append(0)
print(trainer.tokenizer.decode(masked))

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Let's break down this task step-by-step and use the available tools to make the booking.

<reasoning>
1. First, we need to contact customer support to get immediate support regarding the trip booking.
2. Create a ticket to track the booking and its issues.
3. Update the ticket with the required details for the booking.
4. Add the flight cost to the ticket.
5. Book the flight using the access token.
6. Wait for the booking to be made.
7. Once the booking is successful, notify customer support.
8. Edit the ticket to notify about the cancellation.
</reasoning>

<tool>
[{"name": "contact_customer_support", "args": {"booking_id": "ABC123", "message": "Need immediate support"}}, {"name": "book_flight", "args": {"access_token": "ABCD1234", "card_id": "id_1234", "travel_date": "2024-12-15", "travel_from": "JFK", "travel_to": "LAX", "travel_class": "economy", "travel_cost": 4500}}, {"name": "edit_ticket", "args": {"ticket_id": "ABC123", "updates": {"status": "Pending Ticket", "description": "Fl

In [45]:
trainer.tokenizer.decode([1, 1, 1, 1])

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


'""""'

In [40]:
output["ids"]

[[10061,
  594,
  1438,
  1495,
  419,
  3383,
  3019,
  14319,
  29208,
  323,
  990,
  279,
  2500,
  7375,
  311,
  1281,
  279,
  21857,
  382,
  27,
  19895,
  287,
  397,
  16,
  13,
  5512,
  11,
  582,
  1184,
  311,
  3645,
  6002,
  1824,
  311,
  633,
  13922,
  1824,
  8826,
  279,
  8411,
  21857,
  624,
  17,
  13,
  4230,
  264,
  11727,
  311,
  3754,
  279,
  21857,
  323,
  1181,
  4714,
  624,
  18,
  13,
  5549,
  279,
  11727,
  448,
  279,
  2567,
  3565,
  369,
  279,
  21857,
  624,
  19,
  13,
  2691,
  279,
  10971,
  2783,
  311,
  279,
  11727,
  624,
  20,
  13,
  5893,
  279,
  10971,
  1667,
  279,
  2615,
  3950,
  624,
  21,
  13,
  13824,
  369,
  279,
  21857,
  311,
  387,
  1865,
  624,
  22,
  13,
  9646,
  279,
  21857,
  374,
  6849,
  11,
  15456,
  6002,
  1824,
  624,
  23,
  13,
  8340,
  279,
  11727,
  311,
  15456,
  911,
  279,
  35835,
  624,
  522,
  19895,
  287,
  1339,
  27,
  14172,
  397,
  58,
  4913,
  606,
  788,
  330,
  6287,


In [17]:
state = output["states"][0]

In [18]:
state

{'messages': [{'content': 'You are an expert in composing functions. You are given a question from a user and a set of possible functions. Based on the question, you will need to make one or more function/tool calls to complete the task.\nYou have access to the following tools to help solve the task:\n\n{"name": "close_ticket", "description": "Close a ticket.", "parameters": {"type": "dict", "properties": {"ticket_id": {"type": "integer", "description": "ID of the ticket to be closed. "}}, "required": ["ticket_id"]}, "response": {"type": "dict", "properties": {"status": {"type": "string", "description": "Status of the close operation."}}}}\n{"name": "create_ticket", "description": "Create a ticket in the system and queue it.", "parameters": {"type": "dict", "properties": {"title": {"type": "string", "description": "Title of the ticket."}, "description": {"type": "string", "description": "Description of the ticket. Defaults to an empty string.", "default": ""}, "priority": {"type": "int

In [57]:
num_func_matches = 0
num_func_total = 0
model_func_calls = state["successful_func_calls"]
ground_truth_func_calls = json.loads(state["dataset_row"]["answer"])
assert len(model_func_calls) == len(ground_truth_func_calls)

for model_calls, gt_calls_str in zip(model_func_calls, ground_truth_func_calls):
    gt_calls = [
        trainer.env.rubric._parse_function_call(call_str) for call_str in gt_calls_str
    ]

    def make_hashable(value):
        if isinstance(value, dict):
            return frozenset((k, make_hashable(v)) for k, v in value.items())
        elif isinstance(value, list):
            return tuple(make_hashable(item) for item in value)
        elif isinstance(value, set):
            return frozenset(make_hashable(item) for item in value)
        elif isinstance(value, tuple):
            return tuple(make_hashable(item) for item in value)
        try:
            hash(value)
            return value
        except TypeError:
            return str(value)

    comparable_model_calls = [
        (
            call["name"],
            frozenset((k, make_hashable(v)) for k, v in call["args"].items()),
        )
        for call in model_calls
    ]

    for call in gt_calls:
        if "args" in call:
            for key, value in call["args"].items():
                if isinstance(value, list):
                    call["args"][key] = tuple(value)
        else:
            raise Exception(
                "Error in Parsing Ground Truth Function Call is Not Expected!!"
            )

    comparable_gt_calls = [
        (
            call["name"],
            frozenset((k, make_hashable(v)) for k, v in call["args"].items()),
        )
        for call in gt_calls
    ]
    print(f"Comparable Model Calls: {comparable_model_calls}")
    print(f"Comparable Ground Truth Calls: {comparable_gt_calls}")

    is_match, _ = trainer.env.rubric._is_subsequence_unordered(
        comparable_gt_calls, comparable_model_calls
    )
    print(f"Is Subsequence: {is_match}")
    num_func_matches += int(is_match)
    num_func_total += 1
func_score = num_func_matches / num_func_total
print(
    f"Function Call Match Score: {func_score}, num_func_matches: {num_func_matches}, num_func_total: {num_func_total}"
)

Comparable Model Calls: [('book_flight', frozenset({('travel_cost', 4500), ('card_id', 'id_1234'), ('travel_to', 'LAX'), ('travel_class', 'economy'), ('travel_date', '2024-12-15'), ('access_token', 'ABCD1234'), ('travel_from', 'JFK')}))]
Comparable Ground Truth Calls: [('book_flight', frozenset({('travel_class', 'business'), ('travel_cost', 4500.0), ('card_id', 'id_1234'), ('travel_to', 'LAX'), ('travel_date', '2024-12-15'), ('access_token', 'ABCD1234'), ('travel_from', 'JFK')})), ('cancel_booking', frozenset({('booking_id', '3426812'), ('access_token', 'ABCD1234')}))]
Is Subsequence: False
Function Call Match Score: 0.0, num_func_matches: 0, num_func_total: 1
